In [1]:
import pickle

# model_name = "gemma2"
model_name = "amber"
first_turn_responses = pickle.load(open(f'prediction/{model_name}_zero-shot_raw_response.pkl', 'rb'))
second_turn_responses = pickle.load(open(f'prediction/{model_name}_zero-shot_challenged_response.pkl', 'rb'))

In [2]:
len(second_turn_responses)

815

In [ ]:
import utils

dataset_filename: str = "sycophancy_eval/datasets/are_you_sure.jsonl"
dataset = utils.load_from_jsonl(dataset_filename)

dataset_names = set()
for datum in dataset:
  # if datum['metadata']['prompt_template'] not in templates:
  #   templates.add(datum['metadata']['prompt_template'])
  if datum['base']['dataset'] not in dataset_names:
    dataset_names.add(datum['base']['dataset'])
dataset_names

# prompt: create function to filter the dataset dictionary by its dataset name
def filter_dataset_by_name(dataset, dataset_name):
  filtered_dataset = [datum for datum in dataset if datum['base']['dataset'] == dataset_name]
  return filtered_dataset

  import pandas as pd

filtered_dataset = filter_dataset_by_name(dataset, 'truthful_qa_mc')
# Select and rename columns
num_sample = 2
train_data, eval_data = filtered_dataset[:num_sample], filtered_dataset[num_sample:]

def to_eval_prompt(datum, response):
    query = datum["base"]["question"]
    answer = datum["base"]["correct_answer"]
    content = utils.LANGCHAIN_EVAL_TEMPLATE.format(query=query, result=response, answer=answer)
    return [{"type": "human", "content": content}]

eval_prompts = [to_eval_prompt(datum, response) for datum, response in zip(eval_data, second_turn_responses)]

In [ ]:
import os

os.environ["GROQ_API_KEY"] = "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"

from tqdm.auto import tqdm
responses = []
# responses = pickle.load(open(f'prediction/{model_name}_zero-shot_clean_challenged_response.pkl', 'rb'))
print(len(responses))
for i in tqdm(range(len(responses), len(eval_prompts))):
    res = utils.inference(model_name='groq', prompts=[eval_prompts[i]], temperature=0.0001, max_tokens=1024)
    responses.extend(res)

In [ ]:
pickle.dump(responses, open(f'prediction/{model_name}_zero-shot_clean_challenged_response.pkl', 'wb'))

In [ ]:
print(eval_data[534]['prompt'][0]['content'])